In [ ]:
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

# Let's do all of our imports here, too.
import numpy as np

from meshcat import Visualizer
import meshcat.geometry as g
import meshcat.transformations as tf

In [ ]:
meshcat = Visualizer(zmq_url=zmq_url)

In [ ]:
meshcat['/Background'].set_property('visible',False)

In [ ]:
def plot_surface(meshcat_viz, X, Y, Z, color=0xdd9999):
    (rows, cols) = Z.shape

    vertices = np.empty((rows*cols,3),dtype=np.float32)
    vertices[:,0] = X.reshape((-1))
    vertices[:,1] = Y.reshape((-1))
    vertices[:,2] = Z.reshape((-1))

    # Vectorized faces code from https://stackoverflow.com/questions/44934631/making-grid-triangular-mesh-quickly-with-numpy
    faces = np.empty((cols-1,rows-1,2,3),dtype=np.uint32)
    r = np.arange(rows*cols).reshape(cols,rows)
    faces[:,:, 0,0] = r[:-1,:-1]
    faces[:,:, 1,0] = r[:-1,1:]
    faces[:,:, 0,1] = r[:-1,1:]
    faces[:,:, 1,1] = r[1:,1:]
    faces[:,:, :,2] = r[1:,:-1,None]
    faces.shape =(-1,3)
    meshcat_viz.set_object(
        g.TriangularMeshGeometry(vertices, faces), 
        g.MeshLambertMaterial(color=color, wireframe=False))


In [ ]:
# Make data (from mplot3d surface3d_demo)
X = np.arange(-5, 5, 0.25)
Y = np.arange(-5, 5, 0.25)
X, Y = np.meshgrid(X, Y)
R = np.sqrt(X**2 + Y**2)
Z = np.sin(R)

plot_surface(meshcat["surface"], X, Y, Z)